In [ ]:
# import io
# import pandas as pd
# from minio import Minio

# # Create a Minio client
# client = Minio(
#     "localhost:9000",
#     access_key="ROOTUSER",
#     secret_key="CHANGEME123",
#     secure=False  # Set to True if using HTTPS
# )

# bucket_name = "construction-web-scraping"
# prefix = "teduh/"  # Folder where your CSV files are stored

# # List objects with the given prefix
# objects = list(client.list_objects(bucket_name, prefix=prefix, recursive=True))

# # Sort objects by their name (assuming file names contain numeric order like 'data_1.csv', 'data_2.csv', etc.)
# objects.sort(key=lambda obj: obj.object_name)

# dfs = []
# for obj in objects:
#     # Get the object data from MinIO
#     response = client.get_object(bucket_name, obj.object_name)
#     try:
#         # Read the CSV content into a DataFrame using BytesIO
#         df = pd.read_csv(io.BytesIO(response.read()), index_col=0)  # Ensure the index is preserved
#         dfs.append(df)
#     except Exception as e:
#         print(f"Error reading {obj.object_name}: {str(e)}")
#     finally:
#         response.close()
#         response.release_conn()

# # Merge all DataFrames into one while preserving the index order
# if dfs:
#     merged_df = pd.concat(dfs).sort_index()  # Ensures index is sorted correctly
#     print(merged_df.head())
# else:
#     print("No CSV files found to merge.")


    Kod Pemajuan                    Nama Pemaju                 Nama Projek  \
Bil                                                                           
1        30969-1        ASTAKA KIMLUN SDN. BHD.             RESIDENSI ARDEN   
2        30946-1  BLOOMING PROPERTIES SDN. BHD.  RESIDENSI SENIBONG MUTIARA   
3        30937-1                PV M6 SDN. BHD.               THE MAISON 23   
4        30921-1      OYL DEVELOPMENT SDN. BHD.                CYBER AVENUE   
5        30920-1      LWT DEVELOPMENT SDN. BHD.          TAMAN PULAI PERMAI   

    Daerah Projek    Negeri Projek                   No. Permit  \
Bil                                                               
1     Johor Bahru            Johor  30969-1/02-2028/0151(A)-(S)   
2               -            Johor  30946-1/02-2028/0141(N)-(S)   
3               -  Wp Kuala Lumpur  30937-1/02-2027/0165(N)-(L)   
4           Kinta            Perak  30921-1/02-2027/0144(N)-(L)   
5               -           Pahang  30920-1/

In [ ]:
# print(merged_df.shape)

# output_path = "D:\Airflow\construction_web_scrapping\Output folder\Construction.csv"  # Define the local output folder and file name
# merged_df.to_csv(output_path, index=True)  # Save the file with index
# print(f"Merged CSV saved locally at: {output_path}")


(32640, 10)
Merged CSV saved locally at: D:\Airflow\construction_web_scrapping\Output folder\Construction.csv


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MinIO Integration") \
    .getOrCreate()

print("Spark session started successfully!")


Spark session started successfully!


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MinIO Integration") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session started successfully!")


Spark session started successfully!


In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MinIO Integration") \
    .master("local[*]") \
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901"
    ) \
    .getOrCreate()

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "ROOTUSER")
hadoop_conf.set("fs.s3a.secret.key", "CHANGEME123")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

df = spark.read.format("csv") \
    .option("header", "true") \
    .load("s3a://construction-web-scraping/teduh/construction_20250327_batch_10_20.csv")

df.show()


Py4JJavaError: An error occurred while calling o62.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfN

In [ ]:
from pyspark.sql import SparkSession

# # Initialize SparkSession
# spark = SparkSession.builder \
#     .appName("MinIO Integration") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("MinIO Integration") \
    .master("local[*]") \
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901"
    ) \
    .getOrCreate()


# Configure Hadoop to connect to MinIO
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")  # Use HTTPS if applicable
hadoop_conf.set("fs.s3a.access.key", "ROOTUSER")
hadoop_conf.set("fs.s3a.secret.key", "CHANGEME123")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")  # Set to true if using SSL
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


In [10]:
# Replace 'your-bucket' and 'data/yourfile.csv' with your actual bucket name and file path
df = spark.read.format("csv") \
    .option("header", "true") \
    .load("s3a://construction-web-scraping/teduh/construction_20250327_batch_10_20.csv")

df.show()

Py4JJavaError: An error occurred while calling o54.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfN